# 📘 Problem Overview

## 🔍 Steam Cracking Equilibrium via Gibbs Energy Minimization

This problem models the equilibrium composition of a steam cracking reactor effluent at $T = 1000\ \text{K}$ and $P = 1\ \text{atm}$. The feed contains 4 moles of $H_2O$ per mole of $C_2H_6$. The goal is to determine the equilibrium mole fractions of the species present using Gibbs free energy minimization subject to atomic balance constraints.

---

# 📘 Chemical Species and Gibbs Energies

The species considered in the equilibrium mixture are:

| Component | Gibbs Energy of Formation (kcal/mol) |
|-----------|--------------------------------------|
| CH₄       | 4.61                                 |
| C₂H₄      | 28.249                               |
| C₂H₂      | 40.604                               |
| CO₂       | -94.61                               |
| CO        | -47.942                              |
| O₂        | 0                                    |
| H₂        | 0                                    |
| H₂O       | -46.03                               |
| C₂H₆      | 26.13                                |

---

# 📘 Objective Function

We minimize the total Gibbs energy of the system:

$$
G = \sum_{i=1}^{c} n_i \left( G_i + RT \ln \left( \frac{n_i + \varepsilon}{\sum n_i} \right) \right)
$$

Where:
- $n_i$: moles of component $i$
- $G_i$: Gibbs energy of formation of component $i$
- $R$: gas constant (1.9872 cal/mol·K)
- $T$: temperature (1000 K)
- $c$: number of components
- $\varepsilon$: small number to avoid $\ln(0)$

---

# 📘 Atomic Balance Constraints

To ensure conservation of atoms, we impose the following constraints:

### 🔸 Oxygen Balance
$$
g_1 = 2n_{CO_2} + n_{CO} + 2n_{O_2} + n_{H_2O} - 4 = 0
$$

### 🔸 Hydrogen Balance
$$
g_2 = 4n_{CH_4} + 4n_{C_2H_4} + 2n_{C_2H_2} + 2n_{H_2} + 2n_{H_2O} + 6n_{C_2H_6} - 14 = 0
$$

### 🔸 Carbon Balance
$$
g_3 = n_{CH_4} + 2n_{C_2H_4} + 2n_{C_2H_2} + n_{CO_2} + n_{CO} + 2n_{C_2H_6} - 2 = 0
$$

---

# 📘 Python Implementation

In [1]:
import numpy as np
from scipy.optimize import minimize

# Constants
R = 1.9872  # cal/mol·K
T = 1000    # K
epsilon = 1e-20  # small number to avoid log(0)

# Gibbs energies (kcal/mol)
G_kcal = {
    'CH4': 4.61,
    'C2H4': 28.249,
    'C2H2': 40.604,
    'CO2': -94.61,
    'CO': -47.942,
    'O2': 0,
    'H2': 0,
    'H2O': -46.03,
    'C2H6': 26.13
}

# Convert to cal/mol
G_cal = {k: v * 1000 for k, v in G_kcal.items()}

# Initial guesses
initial = np.array([
    0.001,     # CH4
    0.001,     # C2H4
    0.001,     # C2H2
    0.993,     # CO2
    1.0,       # CO
    0.0001,    # O2
    5.992,     # H2
    1.0,       # H2O
    0.001      # C2H6
])

species = ['CH4', 'C2H4', 'C2H2', 'CO2', 'CO', 'O2', 'H2', 'H2O', 'C2H6']

# Objective function
def gibbs_energy(n):
    total = np.sum(n)
    G = 0
    for i, s in enumerate(species):
        G += n[i] * (G_cal[s] + R * T * np.log((n[i] + epsilon) / total))
    return G

# Constraints
def oxygen_balance(n):
    return 2*n[3] + n[4] + 2*n[5] + n[7] - 4

def hydrogen_balance(n):
    return 4*n[0] + 4*n[1] + 2*n[2] + 2*n[6] + 2*n[7] + 6*n[8] - 14

def carbon_balance(n):
    return n[0] + 2*n[1] + 2*n[2] + n[3] + n[4] + 2*n[8] - 2

constraints = [
    {'type': 'eq', 'fun': oxygen_balance},
    {'type': 'eq', 'fun': hydrogen_balance},
    {'type': 'eq', 'fun': carbon_balance}
]

bounds = [(0, None)] * len(species)

# Solve
result = minimize(gibbs_energy, initial, method='SLSQP', bounds=bounds, constraints=constraints)

# Display results
if result.success:
    print("✅ Optimization successful!")
    print(f"Minimum Gibbs Energy: {result.fun:.4f} cal/mol")
    for i, s in enumerate(species):
        print(f"{s}: {result.x[i]:.6f} mol")
else:
    print("❌ Optimization failed:", result.message)

❌ Optimization failed: Positive directional derivative for linesearch


In [4]:
import numpy as np
from scipy.optimize import minimize

# Constants
R = 1.9872  # cal/mol·K
T = 1000    # K
epsilon = 1e-20

# Gibbs energies in kcal/mol → convert to cal/mol
G_kcal = {
    'CH4': 4.61,
    'C2H4': 28.249,
    'C2H2': 40.604,
    'CO2': -94.61,
    'CO': -47.942,
    'O2': 0,
    'H2': 0,
    'H2O': -46.03,
    'C2H6': 26.13
}
G_cal = np.array([v * 1000 for v in G_kcal.values()])
species = list(G_kcal.keys())

# Initial guess (from POLYMATH)
initial_guess = np.array([
    0.001,     # CH4
    0.001,     # C2H4
    0.001,     # C2H2
    0.993,     # CO2
    1.0,       # CO
    0.0001,    # O2
    5.992,     # H2
    1.0,       # H2O
    0.001      # C2H6
])

# Objective function: Gibbs energy
def gibbs_energy(n):
    total = np.sum(n)
    ln_terms = np.log((n + epsilon) / total)
    return np.sum(n * (G_cal + R * T * ln_terms))

# Constraints
def oxygen_balance(n):
    return 2*n[3] + n[4] + 2*n[5] + n[7] - 4

def hydrogen_balance(n):
    return 4*n[0] + 4*n[1] + 2*n[2] + 2*n[6] + 2*n[7] + 6*n[8] - 14

def carbon_balance(n):
    return n[0] + 2*n[1] + 2*n[2] + n[3] + n[4] + 2*n[8] - 2

constraints = [
    {'type': 'eq', 'fun': oxygen_balance},
    {'type': 'eq', 'fun': hydrogen_balance},
    {'type': 'eq', 'fun': carbon_balance}
]

# Bounds: all mole numbers must be non-negative
bounds = [(0, None)] * len(species)

# Solve
result = minimize(gibbs_energy, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)

# Output
if result.success:
    print("✅ Optimization successful!")
    print(f"Minimum Gibbs Energy: {result.fun:.4f} cal/mol")
    for i, s in enumerate(species):
        print(f"{s}: {result.x[i]:.6f} mol")
else:
    print("❌ Optimization failed:", result.message)

❌ Optimization failed: Positive directional derivative for linesearch


In [6]:
import numpy as np

R = 0.00198588 # kcal/mol/K
T = 1000 # K

species = ['CH4', 'C2H4', 'C2H2', 'CO2', 'CO', 'O2', 'H2', 'H2O', 'C2H6']

# $G_^\circ for each species. These are the heats of formation for each
# species.
Gjo = np.array([4.61, 28.249, 40.604, -94.61, -47.942, 0, 0, -46.03, 26.13]) # kcal/mol

In [7]:
import numpy as np

def func(nj):
    nj = np.array(nj)
    Enj = np.sum(nj);
    G = np.sum(nj * (Gjo / R / T + np.log(nj / Enj)))
    return G

In [8]:
Aeq = np.array([[0,   0,    0,   2,   1,  2,  0,  1,   0],      # oxygen balance
                [4,   4,    2,   0,   0,  0,  2,  2,   6],      # hydrogen balance
                [1,   2,    2,   1,   1,  0,  0,  0,   2]])     # carbon balance

# the incoming feed was 4 mol H2O and 1 mol ethane
beq = np.array([4,  # moles of oxygen atoms coming in
                14, # moles of hydrogen atoms coming in
                2]) # moles of carbon atoms coming in

def ec1(n):
    'equality constraint'
    return np.dot(Aeq, n) - beq

def ic1(n):
    '''inequality constraint
       all n>=0
    '''   
    return n

In [11]:
# initial guess suggested in the example
n0 = [1e-3, 1e-3, 1e-3, 0.993, 1.0, 1e-4, 5.992, 1.0, 1e-3] 

n0 = [0.066, 8.7e-08, 2.1e-14, 0.545, 1.39, 5.7e-14, 5.346, 1.521, 1.58e-7]

from scipy.optimize import fmin_slsqp

X = fmin_slsqp(func, n0, f_eqcons=ec1,f_ieqcons=ic1, iter=300, acc=1e-12)

for s,x in zip(species, X):
    print('{0:10s} {1:1.4g}'.format(s, x))

# check that constraints were met
print( np.dot(Aeq, X) - beq)
print( np.all( np.abs( np.dot(Aeq, X) - beq) < 1e-12) )

Singular matrix E in LSQ subproblem    (Exit mode 5)
            Current function value: nan
            Iterations: 2
            Function evaluations: 20
            Gradient evaluations: 2
CH4        0.06691
C2H4       -3.801e-14
C2H2       -1.638e-13
CO2        0.5453
CO         1.388
O2         -4.708e-13
H2         5.345
H2O        1.522
C2H6       3.961e-15
[8.8817842e-16 0.0000000e+00 4.4408921e-16]
True


C:\Users\Mohammad\AppData\Local\Temp\ipykernel_39648\1892264215.py:6: RuntimeWarning: invalid value encountered in log
  G = np.sum(nj * (Gjo / R / T + np.log(nj / Enj)))
